In [17]:
import os
import cv2
import random
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import TensorDataset
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms as transforms

In [18]:
class ImageReader(object):
    def __init__(self):
        self.DATADIR = './PetImages'
        self.categories = ['Dog', 'Cat']
        self.img_width = 56
        self.img_height = 56
    
    def read_images(self):
        self.images = []
        for category in self.categories:
            path = os.path.join(self.DATADIR, category)
            class_num = self.categories.index(category)
            for img in os.listdir(path):
                
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array, (self.img_width, self.img_height))
                    self.images.append([new_array, class_num])
                except:
                    os.remove(os.path.join(path, img))
                    pass
    
    def get_train_and_test(self):
        train_data, test_data = train_test_split(self.images, test_size = 0.2, random_state = 1234)
        X_train = []
        y_train = []
        for element in train_data:
            X_train.append(element[0])
            y_train.append(element[1])
        
        X_train, y_train = np.array(X_train), np.array(y_train)
        X_train = torch.LongTensor(X_train)/255.0
        y_train = torch.LongTensor(y_train)
        train_tensor = TensorDataset(X_train, y_train)

        X_test = []
        y_test = []
        for element in test_data:
            X_test.append(element[0])
            y_test.append(element[1]) 
        
        X_test, y_test = np.array(X_test), np.array(y_test)
        X_test = torch.LongTensor(X_test)/255.0
        y_test = torch.LongTensor(y_test)
        test_tensor = TensorDataset(X_test, y_test)

        return train_tensor, test_tensor

In [19]:
class RN(nn.Module):
    def __init__(self):
        super(RN, self).__init__()
        # We will need 2 convolutional layers, one pooling (twice) and two fully connected
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        # Here we define the real arquitecture
        # Over a conv, pass a relu activation and then pooling
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        # view is used to reshape the tensor x into a 2D tensor of shape 
        x = x.view(-1, 64 * 14 * 14)
        # FC with rely
        x = nn.functional.relu(self.fc1(x))
        # FC without any activation
        x = self.fc2(x)
        return x

In [20]:
Lector = ImageReader()

Lector.read_images()

train, test = Lector.get_train_and_test()

train_loader = DataLoader(train)
test_loader = DataLoader(test)

model = RN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epocas_dict = {}
num_epochs = 10
for epoch in range(num_epochs):
    perdida = 0
    for i, (images, labels) in enumerate(train_loader):
        # Clean gradient before new batch
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Call backward propagation
        loss.backward()
        optimizer.step()
        if (i+1) % 2500 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
        perdida += loss.item()
    epocas_dict[epoch] = perdida/len(train_loader)

print('Loss per epoch:')
for key in epocas_dict:
    print('Epoch {}: {}'.format(key+1, epocas_dict[key]))

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: {} %'.format(100 * correct / total))

Epoch [1/10], Step [2500/19956], Loss: 0.6369
Epoch [1/10], Step [5000/19956], Loss: 0.5877
Epoch [1/10], Step [7500/19956], Loss: 0.8492
Epoch [1/10], Step [10000/19956], Loss: 0.7626
Epoch [1/10], Step [12500/19956], Loss: 0.5438
Epoch [1/10], Step [15000/19956], Loss: 0.0787
Epoch [1/10], Step [17500/19956], Loss: 0.9051
Epoch [2/10], Step [2500/19956], Loss: 0.6428
Epoch [2/10], Step [5000/19956], Loss: 0.0575
Epoch [2/10], Step [7500/19956], Loss: 0.6602
Epoch [2/10], Step [10000/19956], Loss: 0.5484
Epoch [2/10], Step [12500/19956], Loss: 0.4062
Epoch [2/10], Step [15000/19956], Loss: 0.0265
Epoch [2/10], Step [17500/19956], Loss: 0.3711
Epoch [3/10], Step [2500/19956], Loss: 1.3099
Epoch [3/10], Step [5000/19956], Loss: 0.0046
Epoch [3/10], Step [7500/19956], Loss: 0.4589
Epoch [3/10], Step [10000/19956], Loss: 0.4568
Epoch [3/10], Step [12500/19956], Loss: 0.2218
Epoch [3/10], Step [15000/19956], Loss: 0.0386
Epoch [3/10], Step [17500/19956], Loss: 0.1671
Epoch [4/10], Step [25